<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

W tym notebooku przejdziemy całą ścieżkę - od uruchomienia modelu aż po zwrot danych w formacie JSON. Zaczynajmy! :)

### Przygotowanie środowiska

In [ ]:
#%%capture

!pip install accelerate -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q


In [ ]:
import gc
import torch

In [ ]:
import warnings
warnings.filterwarnings("ignore") # tylko na potrzeby naszego demo ;)

### Gated models

W przypadku modeli, na które zostały nałożone zabezpieczenia w postaci kontrolowanego dostępu (gated models), wymagane będzie podanie tokena użytkownika, dostępnego na portalu HuggingFace:

* informacje dotyczące tokenów:
https://huggingface.co/docs/hub/security-tokens
* token użytkownika
https://huggingface.co/settings/tokens

W tym celu należy zainstalować bibliotekę `huggingface_hub[cli]` oraz dokonać logowania za pomocą tokena, co zostało zaprezentowane w poniższych 2 komórkach.

In [ ]:
!pip install -U "huggingface_hub[cli]" -q

In [ ]:
!huggingface-cli login

### Inicjalizacja modelu (4 bit)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer

device = "cuda"
model_name = 'speakleash/Bielik-11B-v2.2-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             quantization_config=quantization_config
                                             )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

### Funkcja pomocnicza do generowania odpowiedzi

In [ ]:
def generate(prompt: str,
             max_tokens: int = 256,
             temperature: float = 0.0,
             top_k: int = 100,
             top_p: int = 1,
             system: str = None,
             return_str: bool = False):

  messages = []

  # Przygotowanie template z promptem
  if system:
    messages.append({"role": "system",
                     "content": system})

  messages.append({"role": "user",
                   "content": prompt})

  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")
  model_inputs = input_ids

  # Przerzucenie inputu na GPU, jeśli jest dostępne
  if torch.cuda.is_available():
    model_inputs = input_ids.to(device)

  # Generowanie odpowiedzi przez model
  outputs = model.generate(
      model_inputs,
      streamer=streamer,
      max_new_tokens=max_tokens,
      do_sample=True if temperature else False,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k)

  # Wydrukowanie odpowiedzi modelu bądź zwrotka do zmiennej
  if return_str:
    decoded = tokenizer.batch_decode(outputs)
    return decoded[0]

#### Test

In [ ]:
# Test z promptem systemowym i poleceniem od użytkownika
temperature = 0.1
max_tokens = 1000

generate(prompt="Hej, czym się zajmujesz?",
         system="Jesteś programistą baz danych, który chce rozwijać swoje kompetencje w obszarze AI",
         temperature=temperature,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Jestem programistą baz danych, ale ostatnio coraz bardziej interesuję się sztuczną inteligencją i chciałbym rozwijać swoje umiejętności w tym kierunku.

### Wynik:

To świetnie, że chcesz rozwijać swoje kompetencje w obszarze AI! Kilka wskazówek, które mogą Ci pomóc:

1. **Nauka podstaw AI**: Zacznij od zrozumienia podstawowych koncepcji AI, takich jak uczenie maszynowe, głębokie uczenie, sieci neuronowe itp. Możesz skorzystać z kursów online, takich jak te oferowane przez Coursera, edX czy Udacity.

2. **Praktyczne doświadczenie**: Znajdź projekty, które pozwolą Ci zastosować teorię w praktyce. Możesz spróbować stworzyć własne modele AI lub dołączyć do społeczności, takich jak Kaggle, gdzie możesz uczestniczyć w konkursach i projektach związanych z AI.

3. **Specjalizacja w dziedzinie baz danych**: Skoro jesteś programistą baz danych, warto skupić się na obszarze AI, który jest związany z przetwarzaniem danych. Na przykład, możesz zainteresować się uczeniem maszynowym w kontekście an

# Przykład 1

Bielik może nadawać się także do analizy sentymentu danej wiadomości, artykułu czy też komentarzy. Sprawdźmy, jak nasz model sobie z tym poradzi!

Wybrałem pierwszy z brzegu artykuł: https://wydarzenia.interia.pl/kraj/news-ekstremalne-warunki-niemal-w-calym-kraju-sypia-sie-alerty,nId,7763701

Najpierw musimy wydobyć tekst z artykułu! Do tego przydadzą się pythonowe biblioteki **requests** oraz **justext**!

In [ ]:
# Instalacja niezbędnych bibliotek
!pip install justext -q

In [ ]:
import requests
import justext

url = "https://wydarzenia.interia.pl/kraj/news-ekstremalne-warunki-niemal-w-calym-kraju-sypia-sie-alerty,nId,7763701"

def get_article_from_url(url: str):
  # Przygotowanie zmiennej do zbierania fragmentów artykułu ze strony internetowej
  article_parts = []

  # Zebranie odpowiedzi w HTML
  response = requests.get(url)

  # Przetworzenie odpowiedzi przy użyciu biblioteki justext
  paragraphs = justext.justext(response.content, justext.get_stoplist("Polish"))
  for paragraph in paragraphs:
    if not paragraph.is_boilerplate:
      article_parts.append(paragraph.text)

  return " ".join(article_parts)

In [ ]:
# Test działania funkcji
get_article_from_url(url)

'Ekstremalne warunki niemal w całym kraju. Sypią się alerty Pogoda w najbliższym czasie nie da odetchnąć. Niedziela będzie kolejnym dniem z tropikalnymi upałami. W wielu miejscach termometry wskażą powyżej 30 stopni Celsjusza. W związku z ekstremalnymi warunkami IMGW wydał ostrzeżenia dla ponad połowy kraju. W lasach obowiązuje bardzo wysokie zagrożenie pożarowe. Fala upałów w Polsce. IMGW wydał ostrzeżenia/Zbyszek Kaczmarek/Agencja FORUM Reklama W niedzielę z nieba znów poleje się żar. Niemal wszędzie notowane będą ponad 30-stopniowe upały. W związku z ekstremalnymi warunkami IMGW wydał ostrzeżenia pierwszego i drugiego stopnia dla połowy kraju. Fala upałów w Polsce. IMGW wydał ostrzeżenia dla połowy kraju W ciągu dnia na wschodzie i południu Polski niebo będzie bezchmurne. Na pozostałym obszarze może wystąpić przelotny deszcz, a lokalnie - także burze. Wiatr, szczególnie nad morzem będzie dość silny. Porywy mogą sięgać do 65 km/h. Reklama Będzie upalnie. Niemal w całej Polsce słupki 

In [ ]:
# Pobranie treści artykułu do zmiennej
article = get_article_from_url(url)

# Przygotowanie promptów
system = 'Jesteś ekspertem od analizy i sentymentu tekstu. Zapoznaj się z treścią podanego artykułu i opisz jego wydźwięk oraz sentyment.'
prompt = f"Tekst do analizy: {article}"

# Zwiększenie limitu tokenów
max_tokens = 1000

# Generowanie odpowiedzi Bielika!
generate(prompt=prompt,
         system=system,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.




Wydźwięk: Negatywny

Sentyment: Negatywny

Uzasadnienie:

1. Wydźwięk:
  - Negatywny: Tekst opisuje ekstremalne warunki pogodowe, takie jak upały, susza i zagrożenie pożarowe, co ma negatywny wpływ na ludzi i środowisko.

2. Sentyment:
  - Negatywny: Sentyment jest negatywny ze względu na opisane zagrożenia i ostrzeżenia wydane przez IMGW, które wskazują na poważne konsekwencje ekstremalnych warunków pogodowych.

3. Dodatkowe uwagi:
  - Tekst zawiera informacje o ostrzeżeniach IMGW, co dodatkowo wzmacnia negatywny wydźwięk i sentyment.
  - Wzmianki o ekstremalnym zagrożeniu pożarowym w lasach oraz ostrzeżenia o suszy hydrologicznej również przyczyniają się do negatywnego sentymentu.
  - Opis upałów i ich wpływu na temperaturę w różnych regionach kraju dodatkowo podkreśla negatywny charakter tekstu.


### Odpowiedź z Areny (dla porównania)

Po dokładnej analizie tekstu, wydźwięk i sentyment tego artykułu można określić jako:

Wydźwięk: Neutralny z tendencją do negatywnego Sentyment: Lekko negatywny

Uzasadnienie:

1. Neutralny wydźwięk:

* Artykuł przekazuje głównie informacje o prognozach pogody i ostrzeżeniach IMGW.
* Nie zawiera wyraźnie pozytywnych lub negatywnych opinii.

2. Tendencja do negatywnego wydźwięku:

* Opisuje ekstremalne warunki pogodowe, które mogą być uciążliwe dla ludzi.
* Wskazuje na zagrożenia związane z upałami, takie jak susza i ryzyko pożarów.

3. Lekko negatywny sentyment:

* Używa słów takich jak "ekstremalne warunki", "ostrzeżenia", "zagrożenie pożarowe", co ma negatywne konotacje.
* Informuje o nieprzyjemnych doświadczeniach związanych z upałami (np. "z nieba znów poleje się żar").
4. Brak wyraźnie pozytywnych elementów:

* Nie ma wzmianek o potencjalnych pozytywnych aspektach upałów (np. możliwości korzystania z plaży czy basenu).

5. Ostrzeżenia i zalecenia:

* Artykuł zawiera wiele ostrzeżeń i zaleceń bezpieczeństwa, co może wywoływać poczucie niepokoju u czytelników.

Podsumowując, artykuł ma głównie charakter informacyjny, ale ze względu na opisywane ekstremalne warunki pogodowe i związane z nimi zagrożenia, jego sentyment jest lekko negatywny.

# Przykład 2

Sprawdźmy ten sam prompt (czyli treść zadania dla modelu), ale tym razem użyjmy innego materiału źródłowego.

In [ ]:
article = "Wasze odestki są wręcz cudowne. Gdyby były niższe byłoby mnie stać na masło, a tak to tylko margaryna i tynk ze ściany na śniadanie."

system = 'Jesteś ekspertem od analizy i sentymentu tekstu. Zapoznaj się z treścią podanego artykułu i opisz jego wydźwięk oraz sentyment.'
prompt = f"Tekst do analizy: {article}"
max_tokens = 1000

generate(prompt=prompt,
         system=system,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.



### Wynik analizy:

Wydźwięk:
1. **Ton satyryczny**: Autor używa ironii i sarkazmu, aby wyrazić swoje niezadowolenie z wysokości odsetek.
2. **Krytyka ekonomiczna**: Tekst odnosi się do sytuacji ekonomicznej, w której wysokie odsetki wpływają negatywnie na codzienne życie.

Sentyment:
1. **Negatywny**: Mimo że autor używa słowa "cudowne" w kontekście odsetek, jest to ironiczne i wyraża niezadowolenie.
2. **Frustracja**: Autor wyraża frustrację z powodu trudnej sytuacji finansowej, która uniemożliwia mu zakup podstawowych produktów.

Podsumowując, tekst ma wydźwięk satyryczny i krytyczny wobec sytuacji ekonomicznej, a sentyment jest negatywny z powodu frustracji i niezadowolenia.


### Odpowiedź z Areny (dla porównania)

Po dokładnej analizie podanego tekstu, można stwierdzić, że wydźwięk i sentyment tego artykułu są mieszane, z przewagą negatywnego sentymentu.

Wydźwięk:

* Pozytywny: Autor wyraża pozytywny stosunek do "odestków" (prawdopodobnie chodzi o odsetki), nazywając je "cudownymi".
* Negatywny: Autor sugeruje, że wysokie odsetki powodują problemy finansowe, uniemożliwiając mu kupno podstawowych produktów spożywczych.

Sentyment:

* Pozytywny: Komplementowanie "odestków" jako "cudownych" może być interpretowane jako pozytywny sentyment wobec tego konkretnego aspektu.
* Negatywny: Opis trudności finansowych i braku możliwości zakupu podstawowych produktów spożywczych wyraża negatywny sentyment wobec sytuacji finansowej autora i potencjalnie wobec wysokich odsetek.

Podsumowując, wydźwięk jest mieszany, ale sentyment jest głównie negatywny, ponieważ autor wyraża niezadowolenie z wpływu wysokich odsetek na jego sytuację finansową.

# Przykład 3

Teraz Bielik zostanie poproszony o nieco bardziej ustrukturyzowaną odpowiedź. Zobaczcie sami...

In [ ]:
article = get_article_from_url(url)

system = 'Jesteś ekspertem od analizy i sentymentu tekstu. Zapoznaj się z treścią podanego artykułu i opisz jego wydźwięk oraz sentyment. Wyniki zwróć w formacie obiektu JSON.'
prompt = f"Tekst do analizy: {article}"
max_tokens = 1000

generate(prompt=prompt,
         system=system,
         max_tokens=max_tokens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32001 for open-end generation.




Wynik analizy:

{
 "wydźwięk": "negatywny",
 "sentiment": "negatywny"
}

Wyjaśnienie:
Artykuł opisuje ekstremalne warunki pogodowe, takie jak tropikalne upały, ostrzeżenia IMGW, zagrożenie pożarowe w lasach oraz suszę hydrologiczną. Wydźwięk i sentyment tekstu są negatywne, ponieważ skupiają się na niekorzystnych i potencjalnie niebezpiecznych aspektach pogody.


### Odpowiedź z Areny (dla porównania)

Na podstawie analizy wydźwięku i sentymentu podanego tekstu, oto wyniki w formacie obiektu JSON:

```json
{
  "Wydźwięk": "Negatywny",
  "Sentyment": "Neutralny",
  "Uzasadnienie": "Tekst opisuje ekstremalne warunki pogodowe, takie jak upały i zagrożenie pożarowe, co ma negatywny wydźwięk. Jednakże, ton artykułu jest głównie informacyjny i ostrzegawczy, co nadaje mu neutralny sentyment."
}
```

Wyjaśnienie krok po kroku:

1. Wydźwięk: Negatywny

* Tekst opisuje ekstremalne warunki pogodowe, takie jak upały powyżej 30 stopni Celsjusza, ostrzeżenia IMGW, zagrożenie pożarowe w lasach i suszę hydrologiczną.
* Te zjawiska pogodowe są zazwyczaj postrzegane jako niekorzystne lub szkodliwe, co nadaje tekstowi negatywny wydźwięk.

2. Sentyment: Neutralny

* Tekst ma głównie charakter informacyjny i ostrzegawczy.
* Autor nie wyraża osobistych emocji ani opinii wobec opisywanych zjawisk.
* Skupia się na przekazaniu faktów i ostrzeżeń, co nadaje tekstowi neutralny sentyment.

3. Uzasadnienie:

* Negatywny wydźwięk wynika z opisu niekorzystnych warunków pogodowych i zagrożeń z nimi związanych.
* Neutralny sentyment wynika z obiektywnego, informacyjnego tonu artykułu, bez wyraźnego emocjonalnego zaangażowania autora.

Sami oceńcie, jak Bielik poradził sobie z postawionymi przed nim zadaniami! :)